In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Google Cloud Set Up

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rastringer/vertex-ai-examples/blob/main/pytorch_on_vertex/gcp_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/rastringer/vertex-ai-examples/blob/main/pytorch_on_vertex/gcp_setup.ipynb.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/github/rastringer/vertex-ai-examples/blob/main/pytorch_on_vertex/gcp_setup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

This notebook has been tested in the following environment:
* Python version = 3.9

#### Overview

This notebook comprises the steps necessary to run the examples in this repository. These include:
* Checking authentication
* Setting up a GCP project
* Enabling Vertex AI and Cloud Storage
* Making a Storage bucket

## GCP Project and billing

**Please follow the steps below to set up and enable billing for a project.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).

2. [Ensure billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

## APIs 

[Enable the Vertex AI](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and [Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com).

## SDK 
(Skip if using Colab/Vertex Workbench)

To run the notebook locally, please install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "<your-project-name>"  # @param {type:"string"}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "<project-region>"  # @param {type: "string"}

#### Authentication

We may need to autnenticate the environment to your GCP account.

**Vertex AI Workbench**
* You are already authenticated, please skip to "Create / set a storage bucket..."

**Local JupyterLab instance:** uncomment and run:

In [ ]:
# ! gcloud auth login

**Colab**, uncomment and run:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**Service account** or other
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

1. In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

2. Click 'Create service account'.

3. In the Service account name field, enter a name, and click Create.

4. In the Grant this service account access to project section, click the Role drop-down list. Type "Vertex AI"

5. Click into the filter box, and select Vertex AI Administrator. Type "Storage Object Admin" 

6. Click into the filter box, and select Storage Object Admin.

7. Click Create. A JSON file that contains your key downloads to your
local environment.

8. Enter the path to your service account key as the
GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''
     

After authenticating to a project, we can set up a Storage bucket and initialize the SDK.

## Create / set a storage bucket to store artifacts.

In [ ]:
# Starts with "gs://..."
BUCKET_URI = "<bucket-uri-unique>"  # @param {type:"string"}

**If** your bucket doesn't already exist, run this cell to create one.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Initialize the Vertex AI SDK for Python.

In [ ]:
from google.cloud import aiplatform as vertexai
try: 
    vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)
    print(f"PROJECT_ID: {PROJECT_ID},\n REGION: {REGION},\n BUCKET_URI: {BUCKET_URI}\n")
    if vertexai:
        print("SDK initialized")
        print(" Ready to go!")
except:
    print("Please repeat the steps above")